Machine Learning para Jornalistas - Aula 2
Esta atividade envolve a criação de um modelo preditivo para eleição de veriadores na cidade de São Paulo a partir de dados disponibilizados pelo TSE.

Esse é o código 'Cópia_de_ML_Jornalismo_Aula_2.ipynb' no Colab.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#os.getcwd()
os.chdir('/home/larissa/repos/2020_brazil_elections/')
diret_trab = os.getcwd()

Importando Dados - bases de dados podem ser encontradas em https://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1


O TSE disponibiliza diversas bases de dados. Para nosso exemplo vamos usar as bases de candidatos, de bens de candidatos e de resultados da eleição.

In [3]:
diret_dados = diret_trab + '/' + 'DATA' + '/'

diret_base_cand = diret_dados + 'consulta_cand_2020' + '/'
diret_base_bens = diret_dados + 'bem_candidato_2020' + '/'
diret_base_result = diret_dados + 'votacao_candidato_munzona_2020' + '/'

sigla_UF = 'MG'
nome_UE = "BELO HORIZONTE"

base_cand = pd.read_csv(diret_base_cand + 'consulta_cand_2020_' + sigla_UF + '.csv', 
                        sep = ';', encoding = 'latin1', decimal = ",")
base_bens = pd.read_csv(diret_base_bens + 'bem_candidato_2020_' + sigla_UF + '.csv', 
                        sep = ';', encoding = 'latin1', decimal = ",")
base_result = pd.read_csv(diret_base_result +'votacao_candidato_munzona_2020_' + sigla_UF + '.csv', 
                          sep = ';', encoding = 'latin1', decimal = ",")

In [ ]:
#list(base_cand.columns)

In [ ]:
#list(base_bens.columns)

In [ ]:
#list(base_result.columns)

Selecionando variáveis da base de candidatos e cidade a ser analisada

In [5]:
candidatos = base_cand[["ANO_ELEICAO","SG_UF","NM_UE","DS_CARGO", "SQ_CANDIDATO", "NM_URNA_CANDIDATO", "CD_SITUACAO_CANDIDATURA",
              "SG_PARTIDO","NR_IDADE_DATA_POSSE", "DS_GENERO", "DS_GRAU_INSTRUCAO", "DS_ESTADO_CIVIL", "DS_COR_RACA",
              "DS_OCUPACAO", "VR_DESPESA_MAX_CAMPANHA", "ST_REELEICAO", "ST_DECLARAR_BENS","ST_CANDIDATO_INSERIDO_URNA"]]


In [6]:
candidatos = candidatos[(candidatos["NM_UE"] == nome_UE) & (candidatos["DS_CARGO"] == "VEREADOR")]

In [7]:
base_result = base_result[(base_result["NM_UE"] == nome_UE) & (base_result["DS_CARGO"] == "Vereador")]

In [8]:
base_bens = base_bens[(base_bens["NM_UE"] == nome_UE)]

Filtrando candidatos aptos

In [10]:
base_cand[['CD_SITUACAO_CANDIDATURA', 'DS_SITUACAO_CANDIDATURA']].drop_duplicates()

,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA
0,12,APTO
23,3,INAPTO


In [11]:
candidatos = candidatos[candidatos["CD_SITUACAO_CANDIDATURA"] == 12]

As bases de bens de candidatos apresentam cada bem declarado em uma linha. Precisamos agregar as inforamções por candidato.Agregando bens por candidato:

In [12]:
bens = base_bens[["SQ_CANDIDATO" ,"VR_BEM_CANDIDATO"]]

bens = bens.groupby(["SQ_CANDIDATO"]).sum()

Os resultados são disponibilizados por seção eleitoral, precisamos agregar os resultados por candidato.

In [13]:
resultados = base_result[["SQ_CANDIDATO", "DS_SIT_TOT_TURNO", "QT_VOTOS_NOMINAIS"]]

In [14]:
resultados_unique = resultados.drop_duplicates(subset = "SQ_CANDIDATO")
resultados_unique = resultados_unique[["SQ_CANDIDATO",  "DS_SIT_TOT_TURNO"]]

#Para criarmos o modelo, precisamos de uma variável resposta binária. Criando variável resposta binária - eleito (1) nao eleito (0)
resultados_unique[['eleito']] = np.where((resultados_unique['DS_SIT_TOT_TURNO'] == 'ELEITO POR MÉDIA') | (resultados_unique['DS_SIT_TOT_TURNO'] == 'ELEITO POR QP'), 1, 0)
#len(resultados_unique.index)
#resultados_unique[['eleito']].sum()
resultados_unique.head(5)

,SQ_CANDIDATO,DS_SIT_TOT_TURNO,eleito
0,130000756903,SUPLENTE,0
7,130000686209,SUPLENTE,0
11,130000757342,SUPLENTE,0
12,130000656940,NÃO ELEITO,0
13,130001013762,SUPLENTE,0


In [15]:
# Soh p verificar se ficou alguem como NA na variável resposta dummy
eleito_null = np.isnan(resultados_unique['eleito']) 
ww = resultados_unique[eleito_null]
len(ww.index)

0

In [16]:
votos = resultados[["SQ_CANDIDATO","QT_VOTOS_NOMINAIS"]]
votos = resultados.groupby(["SQ_CANDIDATO"]).sum()

Juntando todas as bases em uma só

In [17]:
juntando = candidatos.merge(bens, how = 'left', left_on = "SQ_CANDIDATO", right_on = "SQ_CANDIDATO")


In [18]:
juntando = juntando.merge(resultados_unique, how = 'left', left_on = "SQ_CANDIDATO", right_on = "SQ_CANDIDATO")


In [19]:
juntando = juntando.merge(votos, how = 'left', left_on = "SQ_CANDIDATO", right_on = "SQ_CANDIDATO")
juntando['SQ_CANDIDATO']

0       130000633700
1       130001175454
2       130000633714
3       130001042196
4       130000965960
            ...     
1477    130001175473
1478    130001018047
1479    130001013502
1480    130000633722
1481    130001013768
Name: SQ_CANDIDATO, Length: 1482, dtype: int64

Para criarmos o modelo, precisamos de uma variável resposta binária. Criando variável resposta binária - eleito (1) nao eleito (0)

In [20]:
#juntando[['eleito']] = np.where((juntando['DS_SIT_TOT_TURNO'] == 'ELEITO POR MÉDIA') | (juntando['DS_SIT_TOT_TURNO'] == 'ELEITO POR QP'), 1, 0)
#list(juntando.columns)
juntando[['eleito']].sum()

eleito    41.0
dtype: float64

In [21]:
# Casos que os bens do candidato são nulos
VR_BEM_CANDIDATO_null = np.isnan(juntando['VR_BEM_CANDIDATO'])
casos_VR_BEM_CANDIDATO_null = juntando[VR_BEM_CANDIDATO_null]

# Que foram eleitos
casos_eleitos_VR_BEM_CANDIDATO_null = casos_VR_BEM_CANDIDATO_null[casos_VR_BEM_CANDIDATO_null['eleito'] > 0]
casos_eleitos_VR_BEM_CANDIDATO_null

,ANO_ELEICAO,SG_UF,NM_UE,DS_CARGO,SQ_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,...,DS_COR_RACA,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,ST_REELEICAO,ST_DECLARAR_BENS,ST_CANDIDATO_INSERIDO_URNA,VR_BEM_CANDIDATO,DS_SIT_TOT_TURNO,eleito,QT_VOTOS_NOMINAIS
324,2020,MG,BELO HORIZONTE,VEREADOR,130001175469,NIKOLAS FERREIRA,12,PRTB,24,MASCULINO,...,BRANCA,EMPRESÁRIO,692183.02,N,N,SIM,NaN,ELEITO POR QP,1.0,29388.0
549,2020,MG,BELO HORIZONTE,VEREADOR,130001013752,RAMON BIBIANO DA CASA DE APOIO,12,PSD,57,MASCULINO,...,PARDA,VEREADOR,692183.02,S,N,SIM,NaN,ELEITO POR MÉDIA,1.0,5947.0
780,2020,MG,BELO HORIZONTE,VEREADOR,130001116956,ROGERIO ALKIMIM,12,PMN,42,MASCULINO,...,BRANCA,EMPRESÁRIO,692183.02,N,N,SIM,NaN,ELEITO POR QP,1.0,6061.0
978,2020,MG,BELO HORIZONTE,VEREADOR,130000686199,IZA LOURENÇA,12,PSOL,27,FEMININO,...,PRETA,SERVIDOR PÚBLICO FEDERAL,692183.02,N,N,SIM,NaN,ELEITO POR QP,1.0,7771.0
991,2020,MG,BELO HORIZONTE,VEREADOR,130000756931,MARCOS CRISPIM,12,PSC,44,MASCULINO,...,PRETA,OUTROS,692183.02,N,S,SIM,NaN,ELEITO POR MÉDIA,1.0,3355.0
1088,2020,MG,BELO HORIZONTE,VEREADOR,130000757331,GILSON GUIMARÃES,12,REDE,52,MASCULINO,...,PRETA,OUTROS,692183.02,N,S,SIM,NaN,ELEITO POR QP,1.0,3593.0


Dropando linhas com NaN

In [ ]:
#juntando.dropna(subset = ["VR_BEM_CANDIDATO"], inplace = True)
#juntando[['eleito']].sum()

In [22]:
# Se não tem Bens
juntando['VR_BEM_CANDIDATO'] = juntando['VR_BEM_CANDIDATO'].replace(np.nan, 0)
juntando[['eleito']].sum()

eleito    41.0
dtype: float64

In [23]:
VR_BEM_CANDIDATO_null = np.isnan(juntando['VR_BEM_CANDIDATO']) 
xx = juntando[VR_BEM_CANDIDATO_null]
xx

,ANO_ELEICAO,SG_UF,NM_UE,DS_CARGO,SQ_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,...,DS_COR_RACA,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,ST_REELEICAO,ST_DECLARAR_BENS,ST_CANDIDATO_INSERIDO_URNA,VR_BEM_CANDIDATO,DS_SIT_TOT_TURNO,eleito,QT_VOTOS_NOMINAIS


Criando datasets para modelo - variáveis explicativas e variáveis respostas

In [24]:
var_resposta = juntando[['eleito']]

var_explicativas = juntando[['SG_PARTIDO','NR_IDADE_DATA_POSSE','DS_GENERO','DS_GRAU_INSTRUCAO','DS_ESTADO_CIVIL','DS_COR_RACA','ST_REELEICAO','VR_BEM_CANDIDATO']]


In [25]:
var_resposta.isnull().values.any()

True

In [26]:
# Candidatos que nao estao na base_result
eleito_null = juntando[np.isnan(juntando['eleito']) ]
eleito_null

,ANO_ELEICAO,SG_UF,NM_UE,DS_CARGO,SQ_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,...,DS_COR_RACA,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,ST_REELEICAO,ST_DECLARAR_BENS,ST_CANDIDATO_INSERIDO_URNA,VR_BEM_CANDIDATO,DS_SIT_TOT_TURNO,eleito,QT_VOTOS_NOMINAIS
1105,2020,MG,BELO HORIZONTE,VEREADOR,130000757374,JÚLIO FESSÔ,12,REDE,45,MASCULINO,...,PRETA,OUTROS,692183.02,N,N,SIM,0.0,NaN,NaN,NaN


In [ ]:
#aa = base_result[base_result.SQ_CANDIDATO.eq(250000885499)]
#aa

In [27]:
# Excluindo os candidatos que nao estao na base_result
juntando.dropna(subset = ["eleito"], inplace = True)

var_resposta = juntando[['eleito']]

var_explicativas = juntando[['SG_PARTIDO','NR_IDADE_DATA_POSSE','DS_GENERO','DS_GRAU_INSTRUCAO','DS_ESTADO_CIVIL','DS_COR_RACA','ST_REELEICAO','VR_BEM_CANDIDATO']]


In [28]:
var_resposta.isnull().values.any()

False

In [29]:
var_explicativas['SG_PARTIDO'].isnull().values.any()

False

In [30]:
var_explicativas['NR_IDADE_DATA_POSSE'].isnull().values.any()

False

In [31]:
var_explicativas['DS_GENERO'].isnull().values.any()

False

In [32]:
var_explicativas['DS_GRAU_INSTRUCAO'].isnull().values.any()

False

In [33]:
var_explicativas['DS_ESTADO_CIVIL'].isnull().values.any()

False

In [34]:
var_explicativas['DS_COR_RACA'].isnull().values.any()

False

In [35]:
var_explicativas['ST_REELEICAO'].isnull().values.any()

False

In [36]:
var_explicativas['VR_BEM_CANDIDATO'].isnull().values.any()

False

In [37]:
var_explicativas.head()

,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_COR_RACA,ST_REELEICAO,VR_BEM_CANDIDATO
0,PP,44,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),BRANCA,N,1400.00
1,PRTB,34,MASCULINO,SUPERIOR COMPLETO,SOLTEIRO(A),PRETA,N,0.00
2,PP,41,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA,N,405000.00
3,SOLIDARIEDADE,41,FEMININO,SUPERIOR INCOMPLETO,CASADO(A),PARDA,N,0.00
4,AVANTE,48,FEMININO,SUPERIOR COMPLETO,CASADO(A),BRANCA,N,213869.48


In [38]:
var_resposta.head()

,eleito
0,1.0
1,0.0
2,0.0
3,0.0
4,1.0


In [39]:
var_resposta.sum()

eleito    41.0
dtype: float64

Temos diversas variáveis categóricas. Precisamos transformar estas variáveis em numéricas. A variável númerica que assumo valor 1 para uma categoria e 0 para outra é chamada de dummy. Por exemplo - podemos ter uma dummy para se o candidato está tentando a reeleição que vai assumir o valor 1 quano ele estiver disputando a reeleição e 0 quando isso não ocorrer. Gerando Dummies:

In [40]:
var_explicativas_dummies = pd.get_dummies(var_explicativas)

In [41]:
# var_explicativas original
var_explicativas.head()

,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_COR_RACA,ST_REELEICAO,VR_BEM_CANDIDATO
0,PP,44,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),BRANCA,N,1400.00
1,PRTB,34,MASCULINO,SUPERIOR COMPLETO,SOLTEIRO(A),PRETA,N,0.00
2,PP,41,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA,N,405000.00
3,SOLIDARIEDADE,41,FEMININO,SUPERIOR INCOMPLETO,CASADO(A),PARDA,N,0.00
4,AVANTE,48,FEMININO,SUPERIOR COMPLETO,CASADO(A),BRANCA,N,213869.48


In [42]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression()

model = logisticRegr.fit(var_explicativas_dummies, var_resposta.values.ravel())

Criando o modelo a partir da nossa base de treinamento

Criando as previsões a partir do modelo:

In [43]:
predictions = logisticRegr.predict(var_explicativas_dummies)
predictions_prob = logisticRegr.predict_proba(var_explicativas_dummies)


predictions = pd.DataFrame(predictions, columns=['predito'])

predictions_prob = pd.DataFrame(predictions_prob, columns=["Prob_0", "Prob_1"])


In [44]:
predictions_prob.head()

,Prob_0,Prob_1
0,0.971160,0.028840
1,0.938137,0.061863
2,0.957760,0.042240
3,0.963638,0.036362
4,0.977093,0.022907


In [45]:
predictions_prob[['Prob_1']]

,Prob_1
0,0.028840
1,0.061863
2,0.042240
3,0.036362
4,0.022907
...,...
1476,0.061410
1477,0.072099
1478,0.012898
1479,0.021138


In [46]:
predictions.head()

,predito
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


Exportando resultados para analise - exercício de casa

In [47]:
analise = pd.concat([juntando.reset_index(drop = True), predictions_prob], axis = 1)

#analise.to_csv("analisa_sp.csv", sep = ';')


In [48]:
analise.head()

,ANO_ELEICAO,SG_UF,NM_UE,DS_CARGO,SQ_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,...,VR_DESPESA_MAX_CAMPANHA,ST_REELEICAO,ST_DECLARAR_BENS,ST_CANDIDATO_INSERIDO_URNA,VR_BEM_CANDIDATO,DS_SIT_TOT_TURNO,eleito,QT_VOTOS_NOMINAIS,Prob_0,Prob_1
0,2020,MG,BELO HORIZONTE,VEREADOR,130000633700,RUBÃO,12,PP,44,MASCULINO,...,692183.02,N,S,SIM,1400.00,ELEITO POR QP,1.0,3788.0,0.971160,0.028840
1,2020,MG,BELO HORIZONTE,VEREADOR,130001175454,ELIAS DIAS,12,PRTB,34,MASCULINO,...,692183.02,N,N,SIM,0.00,SUPLENTE,0.0,29.0,0.938137,0.061863
2,2020,MG,BELO HORIZONTE,VEREADOR,130000633714,MARCELO DINIZ,12,PP,41,MASCULINO,...,692183.02,N,S,SIM,405000.00,SUPLENTE,0.0,255.0,0.957760,0.042240
3,2020,MG,BELO HORIZONTE,VEREADOR,130001042196,MADRINHA GISLENE LOURDES,12,SOLIDARIEDADE,41,FEMININO,...,692183.02,N,N,SIM,0.00,NÃO ELEITO,0.0,202.0,0.963638,0.036362
4,2020,MG,BELO HORIZONTE,VEREADOR,130000965960,FLÁVIA BORJA,12,AVANTE,48,FEMININO,...,692183.02,N,S,SIM,213869.48,ELEITO POR QP,1.0,5887.0,0.977093,0.022907


In [49]:
# Métricas 
metrics_list = ['min', 'mean', 'median', 'max', 'std']

analise.agg({'Prob_1': metrics_list})

,Prob_1
min,0.000973
mean,0.032990
median,0.022840
max,0.697520
std,0.034476


In [50]:
analise.sort_values(by = "Prob_1", ascending = False)

,ANO_ELEICAO,SG_UF,NM_UE,DS_CARGO,SQ_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,...,VR_DESPESA_MAX_CAMPANHA,ST_REELEICAO,ST_DECLARAR_BENS,ST_CANDIDATO_INSERIDO_URNA,VR_BEM_CANDIDATO,DS_SIT_TOT_TURNO,eleito,QT_VOTOS_NOMINAIS,Prob_0,Prob_1
439,2020,MG,BELO HORIZONTE,VEREADOR,130000965958,DANIEL SIRIEMA,12,AVANTE,48,MASCULINO,...,692183.02,N,S,SIM,12086048.52,SUPLENTE,0.0,3844.0,0.302480,0.697520
30,2020,MG,BELO HORIZONTE,VEREADOR,130001186360,THOMÁS CARRIERE,12,PCB,22,MASCULINO,...,692183.02,N,S,SIM,1329981.92,NÃO ELEITO,0.0,66.0,0.776815,0.223185
1095,2020,MG,BELO HORIZONTE,VEREADOR,130000635838,MELISSA,12,PROS,18,FEMININO,...,692183.02,N,S,SIM,1000.00,SUPLENTE,0.0,88.0,0.808683,0.191317
155,2020,MG,BELO HORIZONTE,VEREADOR,130000635789,BIANCA ANGEL,12,PROS,19,FEMININO,...,692183.02,N,S,SIM,1300.00,SUPLENTE,0.0,2.0,0.820726,0.179274
733,2020,MG,BELO HORIZONTE,VEREADOR,130000635833,KET,12,PROS,19,FEMININO,...,692183.02,N,S,SIM,800.00,SUPLENTE,0.0,4.0,0.820764,0.179236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,2020,MG,BELO HORIZONTE,VEREADOR,130001243009,ALMEIDÃO DO PARTIDÃO,12,PCB,81,MASCULINO,...,692183.02,N,S,SIM,472000.00,NÃO ELEITO,0.0,36.0,0.998142,0.001858
1420,2020,MG,BELO HORIZONTE,VEREADOR,130001021638,DR VICENTE AMORIM,12,PSL,79,MASCULINO,...,692183.02,N,N,SIM,0.00,SUPLENTE,0.0,278.0,0.998185,0.001815
706,2020,MG,BELO HORIZONTE,VEREADOR,130000656874,DONA LURDES,12,PTC,81,FEMININO,...,692183.02,N,S,SIM,300000.00,SUPLENTE,0.0,4.0,0.998260,0.001740
1022,2020,MG,BELO HORIZONTE,VEREADOR,130000757367,BUTINADA,12,REDE,81,MASCULINO,...,692183.02,N,S,SIM,100000.00,SUPLENTE,0.0,176.0,0.998390,0.001610


In [51]:
import altair as alt              # histograma e scatterplot

alt.Chart(analise).mark_point().encode(
        alt.X('Prob_1',
          title = 'Probabilidade de Eleição (output do modelo)'),
        alt.Y('eleito',
          title = 'Eleito (observado nas urnas)')).properties(height = 320, width = 640)

alt.Chart(...)

In [52]:
analise

,ANO_ELEICAO,SG_UF,NM_UE,DS_CARGO,SQ_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,...,VR_DESPESA_MAX_CAMPANHA,ST_REELEICAO,ST_DECLARAR_BENS,ST_CANDIDATO_INSERIDO_URNA,VR_BEM_CANDIDATO,DS_SIT_TOT_TURNO,eleito,QT_VOTOS_NOMINAIS,Prob_0,Prob_1
0,2020,MG,BELO HORIZONTE,VEREADOR,130000633700,RUBÃO,12,PP,44,MASCULINO,...,692183.02,N,S,SIM,1400.00,ELEITO POR QP,1.0,3788.0,0.971160,0.028840
1,2020,MG,BELO HORIZONTE,VEREADOR,130001175454,ELIAS DIAS,12,PRTB,34,MASCULINO,...,692183.02,N,N,SIM,0.00,SUPLENTE,0.0,29.0,0.938137,0.061863
2,2020,MG,BELO HORIZONTE,VEREADOR,130000633714,MARCELO DINIZ,12,PP,41,MASCULINO,...,692183.02,N,S,SIM,405000.00,SUPLENTE,0.0,255.0,0.957760,0.042240
3,2020,MG,BELO HORIZONTE,VEREADOR,130001042196,MADRINHA GISLENE LOURDES,12,SOLIDARIEDADE,41,FEMININO,...,692183.02,N,N,SIM,0.00,NÃO ELEITO,0.0,202.0,0.963638,0.036362
4,2020,MG,BELO HORIZONTE,VEREADOR,130000965960,FLÁVIA BORJA,12,AVANTE,48,FEMININO,...,692183.02,N,S,SIM,213869.48,ELEITO POR QP,1.0,5887.0,0.977093,0.022907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,2020,MG,BELO HORIZONTE,VEREADOR,130001175473,CABO ROMULO SANTIAGO BH,12,PRTB,35,MASCULINO,...,692183.02,N,S,SIM,186334.00,SUPLENTE,0.0,175.0,0.938590,0.061410
1477,2020,MG,BELO HORIZONTE,VEREADOR,130001018047,RENATO DO SALÃO,12,REPUBLICANOS,32,MASCULINO,...,692183.02,N,S,SIM,13500.00,SUPLENTE,0.0,205.0,0.927901,0.072099
1478,2020,MG,BELO HORIZONTE,VEREADOR,130001013502,GILSON REIS,12,PC do B,57,MASCULINO,...,692183.02,S,S,SIM,556129.63,NÃO ELEITO,0.0,3398.0,0.987102,0.012898
1479,2020,MG,BELO HORIZONTE,VEREADOR,130000633722,VAL CONSOLAÇÃO,12,PP,48,FEMININO,...,692183.02,N,S,SIM,1300.00,SUPLENTE,0.0,1518.0,0.978862,0.021138


In [53]:
ponto_corte = 0.9

analise[['pred_eleger']] = np.where(analise[['Prob_1']] >= ponto_corte, 1, 0)

analise[['pred_eleger']].sum()

pred_eleger    0
dtype: int64

In [54]:
y_true = pd.Series(list(map(int, analise['eleito'])), name = 'Actual')
y_pred = pd.Series(list(map(int, analise['pred_eleger'])), name = 'Predicted')


df_confusion = pd.crosstab(y_true, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted,0,All
Actual,,
0,1440,1440
1,41,41
All,1481,1481


In [55]:
# Reeleição
candidatos['ST_REELEICAO'].value_counts()

N    1456
S      26
Name: ST_REELEICAO, dtype: int64